In [61]:
#adapted after https://www.kaggle.com/sentdex/data-science-bowl-2017/first-pass-through-data-w-3d-convnet

#NOT FINISHED
#ZADEVA JE V DELU MA NACELOMA ZDEJ JE SKORI TU TU

import dicom
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import csv
import math

In [64]:
import math

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

def mean(l):
    return sum(l) / len(l)

IMG_PX_SIZE = 150
HM_SLICES = 20

In [75]:
def process_data(patient,img_px_size=50, hm_slices=20):
    
    slices = [dicom.read_file(folder + '/' + s) for s in os.listdir(folder)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    
    #info about picture
    #print(slices[0])
    
    #print array shape
    #print(slices[0].pixel_array.shape, len(slices)) 
    
    #show a slice of choice
    #plt.imshow(slices[125].pixel_array)
    #plt.show()
    
    #resizing to 150*150*20
    new_slices = []

    slices = [cv2.resize(np.array(each_slice.pixel_array),(img_px_size,img_px_size)) for each_slice in slices]

    chunk_sizes = math.ceil(len(slices) / hm_slices)

    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)
        
    if len(new_slices) == hm_slices-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices+2:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val

    if len(new_slices) == hm_slices+1:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val
    
    #showing resized pictures
    #fig = plt.figure()
    #for num,each_slice in enumerate(new_slices):
    #    y = fig.add_subplot(4,5,num+1)
    #    y.imshow(each_slice, cmap='gray')
    #plt.show()
    
    return np.array(new_slices)

In [76]:
data_dir = '../4D-Lung/'
#patient 0
path = data_dir + '100_HM10395/1.3.6.1.4.1.14519.5.2.1.6834.5010.465205689126985052184293614571/'

folders = [x[0] for x in os.walk(path)]
del folders[0]
#folders[:10]

data = []
for num,folder in enumerate(folders[:10]):
    img_data=process_data(folder,img_px_size=IMG_PX_SIZE, hm_slices=HM_SLICES)
    #print(img_data.shape)
    data.append(img_data)
np.save('data-{}-{}-{}.npy'.format(IMG_PX_SIZE,IMG_PX_SIZE,HM_SLICES), data)

In [71]:
def resize_seg(data, img_px_size=150, hm_slices=20):
    new_slices = []
    
    data_seg = []
    for num in range(141):
        #print(data.shape)
        data1 = np.array(data)[:,:,num]
        data_seg.append(cv2.resize(data1,(img_px_size,img_px_size)))

    chunk_sizes = math.ceil(142 / hm_slices)

    for slice_chunk in chunks(data_seg, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)
        
    if len(new_slices) == hm_slices-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices+2:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val

    if len(new_slices) == hm_slices+1:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val
    
    #showing resized pictures
    #fig = plt.figure()
    #for num,each_slice in enumerate(new_slices):
    #    y = fig.add_subplot(4,5,num+1)
    #    y.imshow(each_slice, cmap='gray')
    #plt.show()
    
    return np.array(new_slices)

In [74]:
data_seg_dir = '../patient0-segmented/'

files = [x for x in os.listdir(data_dir)]

print(files)

data_seg = []
for num,file in enumerate(files):
    #print(num)
    seg_data = np.genfromtxt(data_seg_dir + str(num*10) + '-cancer.csv',delimiter=",")
    seg_data = seg_data.reshape((512,512,142))
    #print(seg_data.shape)
    seg_data = resize_seg(seg_data)
    data_seg.append(seg_data)
    
np.save('segdata-{}-{}-{}.npy'.format(IMG_PX_SIZE,IMG_PX_SIZE,HM_SLICES), data_seg)

['0-cancer.csv', '10-cancer.csv', '20-cancer.csv', '30-cancer.csv', '40-cancer.csv', '50-cancer.csv', '60-cancer.csv', '70-cancer.csv', '80-cancer.csv', '90-cancer.csv']
